# **Distribución % de créditos por segmento (trimestre)**

In [7]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "puebla" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_puebla',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2022_puebla',
 'df_financiamientos_2023_puebla',
 'df_financiamientos_2024_puebla']

In [8]:
df_financiamientos_2022_pachuca= df_financiamientos_2022_puebla[['id','mes','monto']]
df_financiamientos_2023_pachuca= df_financiamientos_2023_puebla[['id','mes','monto']]
df_financiamientos_2024_pachuca= df_financiamientos_2024_puebla[['id','mes','monto']]

## 2022

In [9]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2022_pachuca['trimestre'] = df_financiamientos_2022_pachuca['mes'].apply(asignar_trimestre)
df_financiamientos_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2616\3153927988.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_financiamientos_2022_pachuca['trimestre'] = df_financiamientos_2022_pachuca['mes'].apply(asignar_trimestre)


,id,mes,monto,trimestre
0,1,1,3898856.45,1
1,2,1,0.00,1
2,3,1,0.00,1
3,4,1,42231.00,1
4,5,1,81255.00,1
...,...,...,...,...
12158,12159,12,1507500.00,4
12159,12160,12,920000.00,4
12160,12161,12,1641607.36,4
12161,12162,12,0.00,4


In [10]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_pachuca['segmento'] = df_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2022_pachuca.head()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2616\2616155040.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_financiamientos_2022_pachuca['segmento'] = df_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)


,id,mes,monto,trimestre,segmento
0,1,1,3898856.45,1,A
1,2,1,0.00,1,E
2,3,1,0.00,1,E
3,4,1,42231.00,1,E
4,5,1,81255.00,1,E


In [11]:
trimestres_credit_2022_pachuca = df_financiamientos_2022_pachuca[['trimestre','segmento']]
trimestres_credit_2022_pachuca

,trimestre,segmento
0,1,A
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
12158,4,D
12159,4,E
12160,4,D
12161,4,E


In [12]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2022_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2022_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  12163
trimestre          1          2          3          4
segmento                                             
A           0.435748   0.517964   0.452191   0.435748
B           0.945490   0.855052   0.805722   1.035929
C           1.710104   1.882759   1.932089   1.808764
D           4.620571   4.357478   4.316369   4.267039
E          17.684782  16.903724  16.130889  14.059031
ELITE       0.016443   0.016443   0.049330   0.032887
L           0.049330   0.123325   0.098660   0.156211
S           0.863274   1.167475   1.068815   1.200362


In [14]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2022_pachuca)
print("Total de registros: ", total_registros)

# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2022_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns

# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)

# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    xaxis=dict(
        title='Segmentos',  # Título del eje x
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración para desactivar herramientas no deseadas # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2022', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()


Total de registros:  12163
trimestre          1          2          3          4
segmento                                             
A           0.435748   0.517964   0.452191   0.435748
B           0.945490   0.855052   0.805722   1.035929
C           1.710104   1.882759   1.932089   1.808764
D           4.620571   4.357478   4.316369   4.267039
E          17.684782  16.903724  16.130889  14.059031
ELITE       0.016443   0.016443   0.049330   0.032887
L           0.049330   0.123325   0.098660   0.156211
S           0.863274   1.167475   1.068815   1.200362


## 2023

In [15]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2023_pachuca['trimestre'] = df_financiamientos_2023_pachuca['mes'].apply(asignar_trimestre)
df_financiamientos_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2616\1086660624.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,19817.75,1
1,2,1,0.00,1
2,3,1,0.00,1
3,4,1,33908.00,1
4,5,1,277298.00,1
...,...,...,...,...
10832,10833,12,67786.64,4
10833,10834,12,0.00,4
10834,10835,12,0.00,4
10835,10836,12,0.00,4


In [16]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_pachuca['segmento'] = df_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2023_pachuca.head()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2616\1725725462.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,19817.75,1,E
1,2,1,0.00,1,E
2,3,1,0.00,1,E
3,4,1,33908.00,1,E
4,5,1,277298.00,1,E


In [17]:
trimestres_credit_2023_pachuca = df_financiamientos_2023_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
10832,4,E
10833,4,E
10834,4,E
10835,4,E


In [18]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  10837
trimestre          1          2          3          4
segmento                                             
A           0.424472   0.590569   0.525976   0.756667
B           0.904309   0.941220   1.153456   1.245732
C           1.993172   2.030082   2.186952   2.371505
D           4.189351   4.881425   4.992157   5.213620
E          13.869152  15.364031  13.795331  16.775860
ELITE       0.036911   0.046138   0.018455   0.036911
L           0.119959   0.156870   0.147642   0.138415
S           0.987358   1.273415   1.338009   1.494879


In [19]:


# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns
# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)
# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    xaxis=dict(
        title='Segmentos',  # Título del eje x
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración para desactivar herramientas no deseadas
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2023', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()


## 2024

In [20]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2024_pachuca['trimestre'] = df_financiamientos_2024_pachuca['mes'].apply(asignar_trimestre)
df_financiamientos_2024_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2616\1484986024.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,8574319.0,1
1,2,1,1633800.0,1
2,3,1,0.0,1
3,4,1,0.0,1
4,5,1,0.0,1
...,...,...,...,...
6529,6530,7,999999.0,3
6530,6531,7,0.0,3
6531,6532,7,0.0,3
6532,6533,7,0.0,3


In [21]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2024_pachuca['segmento'] = df_financiamientos_2024_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2024_pachuca.head()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_2616\935610015.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,8574319.0,1,S
1,2,1,1633800.0,1,D
2,3,1,0.0,1,E
3,4,1,0.0,1,E
4,5,1,0.0,1,E


In [22]:
trimestres_credit_2023_pachuca = df_financiamientos_2024_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,S
1,1,D
2,1,E
3,1,E
4,1,E
...,...,...
6529,3,E
6530,3,E
6531,3,E
6532,3,E


In [23]:
trimestres_credit_2023_pachuca = trimestres_credit_2023_pachuca[trimestres_credit_2023_pachuca['trimestre']!=3]

In [24]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  5583
trimestre          1          2
segmento                       
A           1.110514   1.343364
B           1.898621   2.471789
C           4.048003   4.746552
D           8.669174  10.245388
E          27.816586  31.954147
ELITE       0.017912   0.161204
L           0.340319   0.232850
S           2.185205   2.758374


In [25]:


# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L", 'ELITE']

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns
# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)

# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    xaxis=dict(
        title='Segmentos',  # Título del eje x
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2024', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()
